In [229]:
from flask import Flask, jsonify, request, render_template

app = Flask(__name__)


@app.route('/intro', methods = ('GET', 'POST'))
def hello():
    return render_template('name.html')


@app.route("/")
def make_read_excel():
    import pandas as pd 
    import numpy as np 
    import json
    from urllib import parse
    import requests
    from sklearn.linear_model import LogisticRegression
    import scipy.stats as ss



    ## 데이터 수집
    apiKey = 'RGAPI-074edd7a-f351-4b1c-ba5e-c61af955eb6b'
    queue = "/RANKED_SOLO_5x5"


    def getMatches(username):
        id = parse.quote(username) # 아이디를 URL 인코딩
        url = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + id +'?api_key=' + apiKey #puuid값을 가져오기 위한 주소
        names = requests.get(url)

        names = names.json()
        puuid = names['puuid'] # 해당 유저의 puuid 값 가져오기

        # 랭크 10경기 매치아이디 가져오기
        n = str(10)
        rankUrl = 'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/' + puuid + '/ids?queue=420&type=ranked&start=0&count='+n+'&api_key='+ apiKey
        r = requests.get(rankUrl)
        r = r.json()
        rankId = r
        return rankId

    def getMatchData(dataFrame):
        # version = '13.9.506.4846'

        for i in range(len(dataFrame)):
            matchid = dataFrame.matches[i]
            url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + matchid +'?api_key=' + apiKey 
            matchresult = requests.get(url)                        
            matchresult = matchresult.json()

            dataFrame.matchdata[i] = pd.json_normalize(matchresult['info']['participants'])
            

    ## n인분 지표 획득
    def final(summonername):
        df = getMatches(summonername)
        summonernameDF = [0] * len(df) 
        matchdata = [0] * len(df) 
        positionDF = [0] * len(df)
        championDF = [0] * len(df)
        winlose = [0] * len(df)

        df = pd.DataFrame([summonernameDF, df, positionDF, championDF, winlose, matchdata]).T
        df.columns = ['summoner', 'matches', 'position', 'champion', 'result', 'matchdata']

        getMatchData(df)

        for i in range(len(df)):
            columns = list(df.matchdata.iloc[i].columns)

            remove_list = ['gameEndedInEarlySurrender', 'gameEndedInSurrender', 'summonerId', 'summonerLevel',\
                        'lane', 'nexusKills', 'nexusLost', 'nexusTakedowns', 'profileIcon', 'puuid', 'riotIdName', 'riotIdTagline', 'role', \
                            'teamEarlySurrendered', 'allInPings', 'baitPings', 'basicPings', 'championId', 'champLevel', 'commandPings', \
                            'dangerPings', 'eligibleForProgression', 'holdPings', \
                            'challenges.dancedWithRiftHerald', 'challenges.hadOpenNexus', 'challenges.outnumberedNexusKill', \
                            'challenges.playedChampSelectPosition', 'challenges.poroExplosions', 'challenges.snowballsHit', \
                            'challenges.takedownsInEnemyFountain', 'perks.statPerks.defense', 'perks.statPerks.flex', 'perks.statPerks.offense',\
                            'perks.styles', 'challenges.damagePerMinute']

            for j in remove_list:
                columns.remove(j)


            df.matchdata.iloc[i] = df.matchdata.iloc[i][columns]
            df.matchdata.iloc[i]['GDM'] = [0] * 10
            df.matchdata.iloc[i]['KPDM'] = [0] * 10
            df.matchdata.iloc[i]['XPDM'] = [0] * 10
            df.matchdata.iloc[i]['GPM'] = [0] * 10
            df.matchdata.iloc[i]['DPM'] = [0] * 10
            df.matchdata.iloc[i]['DTPM'] = [0] * 10

            for j in range(len(df.matchdata.iloc[i])):
                num1 = i // 10 # 몫
                num2 = i % 10 # 나머지 
                opp_index = num1 * 10 + ((num2 + 5) % 10)
                gamelength = df.matchdata.iloc[i]['challenges.gameLength'][j]/60

                df.matchdata.iloc[i]['GPM'][j] = df.matchdata.iloc[i]['goldEarned'][j] / gamelength
                df.matchdata.iloc[i]['DPM'][j] = df.matchdata.iloc[i]['totalDamageDealtToChampions'][j] / gamelength
                df.matchdata.iloc[i]['DTPM'][j] = df.matchdata.iloc[i]['totalDamageTaken'][j] / gamelength
                df.matchdata.iloc[i]['GDM'][j] = (df.matchdata.iloc[i]['goldEarned'][j] -\
                                                df.matchdata.iloc[i]['goldEarned'][opp_index]) / gamelength
                df.matchdata.iloc[i]['KPDM'][j] = (df.matchdata.iloc[i]['kills'][j] + df.matchdata.iloc[i]['assists'][j] - \
                                                df.matchdata.iloc[i]['kills'][opp_index] - df.matchdata.iloc[i]['assists'][opp_index]) / gamelength 
                df.matchdata.iloc[i]['XPDM'][j] = (df.matchdata.iloc[i]['champExperience'][j] - \
                                                df.matchdata.iloc[i]['champExperience'][opp_index]) / gamelength

                # 지금 생각해보니 상당수의 지표들을 분 당 지표로 변환하는게 맞는 것 같다.

            df.matchdata.iloc[i] = df.matchdata.iloc[i][df.matchdata.iloc[i].summonerName == summonername]

        for i in range(len(df)):
            championName = df.matchdata.iloc[i]['championName'].iloc[0]
            position = df.matchdata.iloc[i]['individualPosition'].iloc[0]
            summonerName = df.matchdata.iloc[i]['summonerName'].iloc[0]

            df.position.iloc[i] = position
            df.summoner.iloc[i] = summonerName 
            df.champion.iloc[i] = championName


            try:
                traindata = pd.read_csv(f'./train/{position}/{championName}.csv')  # train data 불러오기
                winrate = pd.read_csv(f'./winrate_predict/{position}/{championName}.csv') # n인분 도출용 데이터 불러오기

            except:
                df.matchdata.iloc[i] = '죄송합니다. 픽률 1% 미만 챔피언입니다.'    # 픽률 1% 이하일 경우
                continue

            for j in ['participantId', 'championName', 'individualPosition', 'teamPosition', 'summonerName',\
                    'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'challenges.mythicItemUsed' ,\
                    'goldEarned', 'challenges.goldPerMinute', 'totalDamageDealtToChampions', 'totalDamageTaken', \
                    'challenges.abilityUses', 'timePlayed', 'assistMePings', 'itemsPurchased', 'pushPings', 'visionClearedPings', \
                    'getBackPings', 'onMyWayPings', 'enemyMissingPings', 'needVisionPings', 'enemyVisionPings', 'challenges.killParticipation']:
                df.matchdata.iloc[i] = df.matchdata.iloc[i].drop(j, axis=1)


            trainX = traindata
            trainX = trainX.drop('win', axis=1)
            trainY = traindata['win']

            testX = df.matchdata.iloc[i].fillna(0)
            testX = testX.drop('win', axis=1)
            testY = df.matchdata.iloc[i]['win']

            columns_testtrain = pd.Series(list(testX.columns) + list(trainX.columns)).drop_duplicates()
            columns_remove = columns_testtrain[len(testX.columns):].to_list() # train <=> test 컬럼 수 차이 나는 거 맞춰주기
            columns_testtrain = columns_testtrain.to_list()

            for j in columns_remove: 
                columns_testtrain.remove(j)

            trainX = trainX[columns_testtrain]

            modelLR = LogisticRegression(random_state=42, penalty='l2', max_iter=1000)
            modelLR.fit(trainX, trainY)

            winrate_predict = pd.DataFrame(modelLR.predict_proba(testX)*100, columns= ['loserate', 'winrate'])['winrate']
            result = pd.concat([winrate, winrate_predict])

            # 승패에 따른 z-score
            if testY.iloc[0] == 1: 
                wins = ss.zscore(result[result.win == 1].winrate).reset_index(drop=True)
                df.matchdata.iloc[i] = wins.iloc[-1]

            elif testY.iloc[0] == 0: 
                loses = ss.zscore(result[result.win == 0].winrate).reset_index(drop=True)
                df.matchdata.iloc[i] = loses.iloc[-1]

            # result2 = result[(result.winrate <= 2.5) & (result.winrate >= -2.5)]
            # # result2 = result[(result.winrate <= -0.05) | (result.winrate >= 0.05)]
            # maximum = df.matchdata.iloc[i]/abs(result2.winrate.max()) # 기대 승률 절댓값들의 최대값

            # 평균은 1이지만, 0 ~ 2인분 범위 안으로 조정

            if df.matchdata.iloc[i] >= 0:
                result_score = round((1 + df.matchdata.iloc[i]/2.5),2)
                if result_score > 2:
                    result_score = 2
                df.matchdata.iloc[i] = str(result_score) + '인분'  

            elif df.matchdata.iloc[i] < 0:
                result_score = round((1 - df.matchdata.iloc[i]/2.5),2)
                if result_score <= 0:
                    result_score = 0.01
                df.matchdata.iloc[i] = str(result_score) + '인분'  

        return df

    # 닉네임을 넣으면 결과가 웹페이지에서 데이터프레임으로 출력.
    df = final("Hide on bush")

    return df.to_html()
    


if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.30.1.57:5000
Press CTRL+C to quit
C:\Users\PenguinKim\AppData\Local\Temp\ipykernel_27756\295357558.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.matchdata.iloc[i]['GPM'][j] = df.matchdata.iloc[i]['goldEarned'][j] / gamelength
C:\Users\PenguinKim\AppData\Local\Temp\ipykernel_27756\295357558.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.matchdata.iloc[i]['DPM'][j] = df.matchdata.iloc[i]['totalDamageDealtToChampions'][j] / gamelength
C:\Users\PenguinKim\AppData\Local\Temp\ipykernel_27756\295357558.py:101